In [1]:
%reload_ext autoreload
%autoreload 2

from vrAnalysis.session import vrExperiment, redCellProcessing
from vrAnalysis.redgui import redCellGUI as rgui
from vrAnalysis2.database import get_database
from vrAnalysis2.sessions import B2Session
sessiondb = get_database("vrSessions")

In [ ]:
# Below is the "pipeline" for each mouse

In [22]:
# Start by checking which if any red cell parameters haven't been saved in any sessions
# (Note - this is using my original GUI rather than cellector style processing)
for ses in sessiondb.iter_sessions(imaging=True):
    saved_one = [str(so.name) for so in ses.get_saved_one()]
    saved_one_cutoffs = [so for so in saved_one if "minMaxCutoff" in so]
    params = [so[so.find("Red"):so.find(".")] for so in saved_one_cutoffs]
    if len(params) > 0:
        params_print = ", ".join([str(so) for so in params])
    else:
        params_print = "<<<<<<<<<<<<<<<<< NONE FOUND!!!!! >>>>>>>>>>>>>>>>>>>"
        print(ses, params_print)

In [ ]:
# Get the sessions for a given mouse (based on which mice haven't been processed from above)
ises = sessiondb.iter_sessions(imaging=True, mouseName="ATL022", dontTrack=False)
for ses in ises:
    print(ses)

In [15]:
# choose a session by picking the mouse name, datestring, and session ID.
session = ises[2]
mouseName = session.mouse_name
dateString = session.date
sessionid = session.session_id

In [ ]:
# Load the experiment object, red cell processing object, and open the GUI
vrexp = vrExperiment(mouseName, dateString, sessionid)
redCell = redCellProcessing(vrexp) # create red cell processing object
redSelection = rgui.redSelectionGUI(redCell) # open the GUI

In [ ]:
# Copy criterion from one session to another
copyCriterionFrom = redCellProcessing(mouseName, dateString, sessionid, autoload=False)
for ses in sessiondb.iterSessionNeedRedCellQC(mouseName=mouseName):
    redCell = redCellProcessing(ses, autoload=False)
    redCell.updateFromSession(copyCriterionFrom)